In [1]:
# !pip install Prophet

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from prophet import Prophet
from tqdm import tqdm
import logging
from holidays import CountryHoliday
import requests
import optuna 
from sklearn.metrics import make_scorer


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


In [3]:
logging.getLogger('prophet').disabled = True
logging.getLogger('cmdstanpy').disabled = True
logging.getLogger('prophet').setLevel(logging.WARNING) 

In [4]:
train = pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv", index_col="id")
test = pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv", index_col="id")
sample_sub = pd.read_csv("/kaggle/input/playground-series-s3e19/sample_submission.csv")

In [5]:
train.head()

,date,country,store,product,num_sold
id,,,,,
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      136950 non-null  object
 1   country   136950 non-null  object
 2   store     136950 non-null  object
 3   product   136950 non-null  object
 4   num_sold  136950 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 6.3+ MB


In [7]:
train.describe()

,num_sold
count,136950.000000
mean,165.522636
std,183.691575
min,2.000000
25%,46.000000
50%,98.000000
75%,184.000000
max,1380.000000


In [8]:
train.isnull().sum()

date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [9]:
train.columns

Index(['date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [10]:
print(np.unique(train["product"], return_counts=True))
print(np.unique(train["country"], return_counts=True))
print(np.unique(train["store"], return_counts=True))

(array(['Using LLMs to Improve Your Coding',
       'Using LLMs to Train More LLMs',
       'Using LLMs to Win Friends and Influence People',
       'Using LLMs to Win More Kaggle Competitions',
       'Using LLMs to Write Better'], dtype=object), array([27390, 27390, 27390, 27390, 27390]))
(array(['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain'], dtype=object), array([27390, 27390, 27390, 27390, 27390]))
(array(['Kagglazon', 'Kaggle Learn', 'Kaggle Store'], dtype=object), array([45650, 45650, 45650]))


In [11]:
train.date.nunique()

1826

In [12]:
train['type'] = "train"
test['type'] = "test"

In [13]:
full_data = pd.concat([train, test], axis=0, keys=('train','test'))

In [14]:
full_data

date    country         store  \
      id                                            
train 0       2017-01-01  Argentina  Kaggle Learn   
      1       2017-01-01  Argentina  Kaggle Learn   
      2       2017-01-01  Argentina  Kaggle Learn   
      3       2017-01-01  Argentina  Kaggle Learn   
      4       2017-01-01  Argentina  Kaggle Learn   
...                  ...        ...           ...   
test  164320  2022-12-31      Spain     Kagglazon   
      164321  2022-12-31      Spain     Kagglazon   
      164322  2022-12-31      Spain     Kagglazon   
      164323  2022-12-31      Spain     Kagglazon   
      164324  2022-12-31      Spain     Kagglazon   

                                                     product  num_sold   type  
      id                                                                       
train 0                    Using LLMs to Improve Your Coding      63.0  train  
      1                        Using LLMs to Train More LLMs      66.0  train  
      2       Using LLMs to Win Friends and Influence People       9.0  train  
      3           Using LLMs to Win More Kaggle Competitions      59.0  train  
      4                           Using LLMs to Write Better      49.0  train  
...                                                      ...       ...    ...  
test  164320               Using LLMs to Improve Your Coding       NaN   test  
      164321                   Using LLMs to Train More LLMs       NaN   test  
      164322  Using LLMs to Win Friends and Influence People       NaN   test  
      164323      Using LLMs to Win More Kaggle Competitions       NaN   test  
      164324                      Using LLMs to Write Better       NaN   test  

[164325 rows x 6 columns]

In [15]:
def extractDate(df):
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['dayofmonth'] = df.date.dt.day
    df['dayofweek'] = df.date.dt.dayofweek
    df['dayofyear'] = df.date.dt.dayofyear
    df['weekofmonth'] = (df.date.dt.day - 1) // 7 + 1
extractDate(full_data)

In [16]:
full_data

date    country         store  \
      id                                           
train 0      2017-01-01  Argentina  Kaggle Learn   
      1      2017-01-01  Argentina  Kaggle Learn   
      2      2017-01-01  Argentina  Kaggle Learn   
      3      2017-01-01  Argentina  Kaggle Learn   
      4      2017-01-01  Argentina  Kaggle Learn   
...                 ...        ...           ...   
test  164320 2022-12-31      Spain     Kagglazon   
      164321 2022-12-31      Spain     Kagglazon   
      164322 2022-12-31      Spain     Kagglazon   
      164323 2022-12-31      Spain     Kagglazon   
      164324 2022-12-31      Spain     Kagglazon   

                                                     product  num_sold   type  \
      id                                                                        
train 0                    Using LLMs to Improve Your Coding      63.0  train   
      1                        Using LLMs to Train More LLMs      66.0  train   
      2       Using LLMs to Win Friends and Influence People       9.0  train   
      3           Using LLMs to Win More Kaggle Competitions      59.0  train   
      4                           Using LLMs to Write Better      49.0  train   
...                                                      ...       ...    ...   
test  164320               Using LLMs to Improve Your Coding       NaN   test   
      164321                   Using LLMs to Train More LLMs       NaN   test   
      164322  Using LLMs to Win Friends and Influence People       NaN   test   
      164323      Using LLMs to Win More Kaggle Competitions       NaN   test   
      164324                      Using LLMs to Write Better       NaN   test   

              year  month  dayofmonth  dayofweek  dayofyear  weekofmonth  
      id                                                                  
train 0       2017      1           1          6          1            1  
      1       2017      1           1          6          1            1  
      2       2017      1           1          6          1            1  
      3       2017      1           1          6          1            1  
      4       2017      1           1          6          1            1  
...            ...    ...         ...        ...        ...          ...  
test  164320  2022     12          31          5        365            5  
      164321  2022     12          31          5        365            5  
      164322  2022     12          31          5        365            5  
      164323  2022     12          31          5        365            5  
      164324  2022     12          31          5        365            5  

[164325 rows x 12 columns]

In [17]:
def getholidays(df):
    years = np.arange(df.year.min(), df.year.max()+1)
    CountryList = ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain']
    holidays = pd.DataFrame(columns = ['date','holiday','country'])
    
    common_holidays = pd.DataFrame({
    'date': pd.to_datetime(['2017-04-16','2017-12-24', '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29', '2017-12-30', '2017-12-31',
                            '2018-04-01','2018-12-24', '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29', '2018-12-30', '2018-12-31',
                            '2019-04-21','2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31',
                            '2020-04-12','2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31',
                            '2021-04-04','2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
                            '2022-04-17','2022-12-24', '2022-12-25', '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29', '2022-12-30', '2022-12-31']),
    'holiday': 'Holiday Season'
    })
    
    for country in CountryList:
        for h in CountryHoliday(country=country, years=years).items():
            i = len(holidays)
            holidays.loc[i,'date'] = h[0]
            holidays.loc[i,'holiday'] = h[1]
            holidays.loc[i,'country'] = country
        
        common_holidays['country'] = country
        holidays = pd.concat([holidays, common_holidays], ignore_index=True)
    
    holidays['date'] = pd.to_datetime(holidays['date'])
    holidays['isHoliday'] = 1
    df = pd.merge(df, holidays, how="left", on=['date', 'country'])
    df['holiday'] = df['holiday'].fillna("Not Holiday")
    df['isHoliday'] = df['isHoliday'].fillna(0)
    
    df['Friday'] = (df['dayofweek'] == 4).astype(int)
    df['Saturday'] = (df['dayofweek'] == 5).astype(int)
    df['Sunday'] = (df['dayofweek'] == 6).astype(int)
    
    #Quarter, weekly season
    df['Q1'] = ((df['month'] >= 1) & (df['month'] <= 3)).astype(int)
    df['Q2'] = ((df['month'] >= 4) & (df['month'] <= 6)).astype(int)
    df['Q3'] = ((df['month'] >= 7) & (df['month'] <= 9)).astype(int)
    df['Q4'] = ((df['month'] >= 10) & (df['month'] <= 12)).astype(int)
    
    df = df.drop_duplicates(subset=['date', 'country','product','store'], keep='first')
    
    return df, holidays

In [18]:
full_data, Holidays = getholidays(full_data)
Holidays = Holidays.rename(columns={'date':'ds'})
Holidays['ds']=pd.to_datetime(Holidays['ds'])

In [19]:
full_data

,date,country,store,product,num_sold,type,year,month,dayofmonth,dayofweek,...,weekofmonth,holiday,isHoliday,Friday,Saturday,Sunday,Q1,Q2,Q3,Q4
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0,train,2017,1,1,6,...,1,New Year's Day,1.0,0,0,1,1,0,0,0
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0,train,2017,1,1,6,...,1,New Year's Day,1.0,0,0,1,1,0,0,0
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0,train,2017,1,1,6,...,1,New Year's Day,1.0,0,0,1,1,0,0,0
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59.0,train,2017,1,1,6,...,1,New Year's Day,1.0,0,0,1,1,0,0,0
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49.0,train,2017,1,1,6,...,1,New Year's Day,1.0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165130,2022-12-31,Spain,Kagglazon,Using LLMs to Improve Your Coding,NaN,test,2022,12,31,5,...,5,Holiday Season,1.0,0,1,0,0,0,0,1
165131,2022-12-31,Spain,Kagglazon,Using LLMs to Train More LLMs,NaN,test,2022,12,31,5,...,5,Holiday Season,1.0,0,1,0,0,0,0,1
165132,2022-12-31,Spain,Kagglazon,Using LLMs to Win Friends and Influence People,NaN,test,2022,12,31,5,...,5,Holiday Season,1.0,0,1,0,0,0,0,1
165133,2022-12-31,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,NaN,test,2022,12,31,5,...,5,Holiday Season,1.0,0,1,0,0,0,0,1


In [20]:
def covid(df):
    df['pre_covid'] = np.where(df.date<'2020-04-01',0,1)
    df['post_covid'] = np.where(df.date>'2020-07-01',0,1)
    df['covid'] = np.where(((df.date>'2020-04-01')& (df.date<'2020-07-01')),0,1)
    return df
full_data = covid(full_data)

In [21]:
def seasonality_features(df):
    df['month_sine'] = np.sin(2*np.pi*df.month/df.month.max())
    df['month_cosine'] = np.cos(2*np.pi*df.month/df.month.max())
    df['day_sine'] = np.sin(2*np.pi*df.dayofmonth/df.dayofmonth.max())
    df['day_cosine'] = np.cos(2*np.pi*df.dayofmonth/df.dayofmonth.max())
    
    return df
full_data = seasonality_features(full_data)

In [22]:
def getgdp(df):
    url = "https://api.worldbank.org/v2/country/{0}/indicator/NY.GDP.PCAP.CD?date={1}:{2}&format=json"
    country_codes = ['ARG', 'CAN', 'EST', 'JPN', 'ESP']  # Country codes
    countries = ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain']  # Original country names
    start_date = "2017"
    end_date = "2022"
    
    gdp = pd.DataFrame(columns=['year','country','gdp'])
    for code, country in zip(country_codes, countries):
        api_url = url.format(code,start_date,end_date)
        response = requests.get(api_url)
        data = response.json()[1]
        for item in data:
            i = len(gdp)
            gdp.loc[i, 'year'] = item['date']
            gdp.loc[i, 'gdp'] = item['value']
            gdp.loc[i, 'country'] = country
            
    gdp['year']=gdp['year'].astype(int)
    gdp['country'] = gdp['country'].astype(str)
    
    merged_df = pd.merge(df, gdp, how="left", on=['year', 'country'])
    return merged_df

full_data = getgdp(full_data)

In [23]:
full_data.isna().sum()

date                0
country             0
store               0
product             0
num_sold        27375
type                0
year                0
month               0
dayofmonth          0
dayofweek           0
dayofyear           0
weekofmonth         0
holiday             0
isHoliday           0
Friday              0
Saturday            0
Sunday              0
Q1                  0
Q2                  0
Q3                  0
Q4                  0
pre_covid           0
post_covid          0
covid               0
month_sine          0
month_cosine        0
day_sine            0
day_cosine          0
gdp                 0
dtype: int64

In [24]:
# lets change the names of the values because they are too big
def df_transformations(df):    
    df['store'] = df['store'].replace(['Kagglazon', 'Kaggle Learn', 'Kaggle Store'],['Kazon', 'Klearn', 'Kstore'])
    df['product'] = df['product'].replace(['Using LLMs to Improve Your Coding',
            'Using LLMs to Train More LLMs',
            'Using LLMs to Win Friends and Influence People',
            'Using LLMs to Win More Kaggle Competitions',
            'Using LLMs to Write Better'],['Improve', 'Train', 'WinFriends', 'WinMore', 'Write'])
    df['CSP'] = df['country']+'_'+df['store']+'_'+df['product']
df_transformations(full_data)

In [25]:
def createlagfeatures(df):
    lag_features = ["num_sold"]
    exogeneous_features = []
    window_size = 75
    
    for feature in lag_features:
        df[feature+"_lag1"] = df[feature].shift(365)
        exogeneous_features.append(feature+"_lag1")
        df[feature+"_lag2"] = df[feature].shift(365*2)
        exogeneous_features.append(feature+"_lag2")
    
    df = df.fillna(method="bfill")
    df = df.fillna(method="ffill")   
    return df

full_data = createlagfeatures(full_data)

In [26]:
full_data.columns

Index(['date', 'country', 'store', 'product', 'num_sold', 'type', 'year',
       'month', 'dayofmonth', 'dayofweek', 'dayofyear', 'weekofmonth',
       'holiday', 'isHoliday', 'Friday', 'Saturday', 'Sunday', 'Q1', 'Q2',
       'Q3', 'Q4', 'pre_covid', 'post_covid', 'covid', 'month_sine',
       'month_cosine', 'day_sine', 'day_cosine', 'gdp', 'CSP', 'num_sold_lag1',
       'num_sold_lag2'],
      dtype='object')

In [27]:
train_df = full_data[full_data['type']=="train"]
test_df = full_data[full_data['type']=="test"]
train_df.drop(columns= ['type'], inplace=True)
test_df.drop(columns= ['type'], inplace=True)

/tmp/ipykernel_32/3026254760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns= ['type'], inplace=True)
/tmp/ipykernel_32/3026254760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns= ['type'], inplace=True)


In [28]:
def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    num = np.abs(y_pred-y_true)
    den = (np.abs(y_true)+ np.abs(y_pred))
    return (np.sum((2*num)/den)*100)/len(y_true)
smape_scorer = make_scorer(smape, greater_is_better=False)

In [29]:
train_df.columns

Index(['date', 'country', 'store', 'product', 'num_sold', 'year', 'month',
       'dayofmonth', 'dayofweek', 'dayofyear', 'weekofmonth', 'holiday',
       'isHoliday', 'Friday', 'Saturday', 'Sunday', 'Q1', 'Q2', 'Q3', 'Q4',
       'pre_covid', 'post_covid', 'covid', 'month_sine', 'month_cosine',
       'day_sine', 'day_cosine', 'gdp', 'CSP', 'num_sold_lag1',
       'num_sold_lag2'],
      dtype='object')

In [ ]:
def optimization_prophet(trial):
    params = {
        'seasonality_prior_scale': trial.suggest_float('seasonality_prior_scale', 1, 20 ),
        'holidays_prior_scale': trial.suggest_float('holidays_prior_scale', 0.1, 10),
        'changepoint_prior_scale': trial.suggest_float('changepoint_prior_scale', 0.01, 0.5),
        'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
    }
    scores = []
    for CSP in np.unique(train_df['CSP']):
        
        country = train_df['country'][train_df['CSP']==CSP].unique()[0]
        holidays_df = Holidays[['ds', 'holiday']].loc[Holidays['country']==country]
        train_cv = train_df.loc[(train_df['CSP'] == CSP) & (train_df['date']<='2021-06-01')].rename(columns={'date':'ds', 'num_sold':'y'})
        val_cv = train_df.loc[(train_df['CSP'] == CSP) & (train_df['date']>'2021-06-01')].rename(columns={'date':'ds', 'num_sold':'y'})

        prophet = Prophet(weekly_seasonality=False,
          yearly_seasonality=False,
          interval_width=0.95,
          holidays = holidays_df,
                     **params)
                
        prophet.add_regressor('num_sold_lag1')
        prophet.add_regressor('num_sold_lag2')
        prophet.add_regressor('gdp')
        prophet.add_regressor('covid')

        prophet.add_seasonality(
            name='yearly_pre_covid',
            period=365,
            fourier_order=10,
            condition_name='pre_covid',
        )
        prophet.add_seasonality(
            name='yearly_post_covid',
            period=365,
            fourier_order=10,
            condition_name='post_covid',
        )
        prophet.add_seasonality(
            name='weekly_pre_covid',
            period=7,
            fourier_order=3,
            condition_name='pre_covid',
        )
        prophet.add_seasonality(
            name='weekly_post_covid',
            period=7,
            fourier_order=3,
            condition_name='post_covid',
        );
        prophet.fit(train_cv)
        forecast = prophet.predict(val_cv)

        # Calculate mean absolute error
        score = smape(val_cv['y'], forecast['yhat'])
        scores.append(score)
          
    return np.mean(scores)

study = optuna.create_study(direction="minimize")
study.optimize(optimization_prophet, n_trials=10)
best_params = study.best_params
best_score = study.best_value
print("Best Parameters: ", best_params)
print("Best Score: ", best_score)

[I 2023-08-23 15:23:18,568] A new study created in memory with name: no-name-885c7928-cc51-46af-8b01-d9729b2a2882
[I 2023-08-23 15:25:52,518] Trial 0 finished with value: 10.472719736336266 and parameters: {'seasonality_prior_scale': 16.900485102581744, 'holidays_prior_scale': 0.23589119510397336, 'changepoint_prior_scale': 0.37826259446760296, 'seasonality_mode': 'additive'}. Best is trial 0 with value: 10.472719736336266.
[I 2023-08-23 15:28:18,701] Trial 1 finished with value: 10.45640049200888 and parameters: {'seasonality_prior_scale': 11.137725102161465, 'holidays_prior_scale': 4.156483826614669, 'changepoint_prior_scale': 0.3205159638349535, 'seasonality_mode': 'additive'}. Best is trial 1 with value: 10.45640049200888.


In [ ]:
for CSP in np.unique(train_df['CSP']):
        
        country = train_df['country'][train_df['CSP']==CSP].unique()[0]
        holidays_df = Holidays[['ds', 'holiday']].loc[Holidays['country']==country]
        train_cv = train_df.loc[train_df['CSP'] == CSP].rename(columns={'date':'ds', 'num_sold':'y'})
        test_cv = test_df.loc[test_df['CSP'] == CSP].rename(columns={'date':'ds', 'num_sold':'y'})

        params = best_params
        
        prophet = Prophet(weekly_seasonality=False,
          yearly_seasonality=False,
          interval_width=0.95,
          holidays = holidays_df,
                     **params)
                
        prophet.add_regressor('num_sold_lag1')
        prophet.add_regressor('num_sold_lag2')
        prophet.add_regressor('gdp')
        prophet.add_regressor('covid')

        prophet.add_seasonality(
            name='yearly_pre_covid',
            period=365,
            fourier_order=10,
            condition_name='pre_covid',
        )
        prophet.add_seasonality(
            name='yearly_post_covid',
            period=365,
            fourier_order=10,
            condition_name='post_covid',
        )
        prophet.add_seasonality(
            name='weekly_pre_covid',
            period=7,
            fourier_order=3,
            condition_name='pre_covid',
        )
        prophet.add_seasonality(
            name='weekly_post_covid',
            period=7,
            fourier_order=3,
            condition_name='post_covid',
        );
        prophet.fit(train_cv)
        forecast = prophet.predict(test_cv)
        test_df.loc[test_df['CSP'] == CSP, 'num_sold'] = forecast['yhat'].values

In [ ]:
test_df.isnull().sum()

In [ ]:
sample_sub['num_sold'][sample_sub.id==test_df.index] = test_df['num_sold'][test_df.index==sample_sub.id].values

In [ ]:
sample_sub

In [ ]:
sample_sub.to_csv("submission.csv", index=False)